# Correlation Exercise – *Galileo E1‑C & GPS L1 C/A*

This interactive notebook guides you through the **coarse‑acquisition** step used by every GNSS receiver.  You’ll fill in the missing pieces, run the code, and **acquire a real satellite you recorded**.

## How to work in this notebook
1. **Read the short theory sections**  They recap what the code does. Also recall the presentation slides.
2. **Open each *Task* cell (yellow bar on the left) and replace the `...` with real code or numbers.**  Run the cell *(Shift‑Enter)*.
3. **Run the script cell that follows.**  If something is still missing the script will stop and tell you what.
4. Move on to the next section.

👉 **Tip 1:** execute the notebook **top‑to‑bottom** the first time so that all variables are visible where the scripts expect them.

👉 **Tip 2:** *Click regularly on the "save to browser storage button" (next to the download button) to not lose your progress.*

# 0. IQ Data and Pre-Processing
This section demonstrates how to transform raw **I/Q** recordings into a cleaned MATLAB `.mat` file that later acquisition steps can load.

**You will:**
1. Set the sampling frequency and file path  
2. Read the binary file, format the complex samples  
3. Inspect spectrum, time‑domain traces and amplitude histogram  
4. Save the processed vector as **`IQ_samples.mat`**

Follow the numbered code cells below and execute them in order (⇧ + ⏎).

### 1.  Set Sampling Frequency

In [ ]:
% Sampling frequency (Hz)
fs = _______;  % ← Enter sampling frequency, e.g. 1.000e6 for 1 MHz

### 2.  Set Data File Path

In [ ]:
% Path to I/Q data file
data_file = '________.dat';  % ← Enter filename, e.g. 'IQ.dat'

### 3.  Load and Format I/Q Samples
Read the binary file, handle interleaved I/Q channels, and remove initial transients.

In [ ]:
% === Fixed Parameters ===
fi = 0;                   % Intermediate frequency (Hz)
IsComplex = true;         % True if data is I/Q
Is8Bits = false;          % 8-bit integer format
Is16Bits = false;         % 16-bit integer format
Is32BitsFloat = true;     % 32-bit float format
secondOfData = 1;         % Seconds of data to read
fl1 = 1575.42e6;          % Center frequency (Hz) for labels
% ========================

% Read binary samples
[fid, errmsg] = fopen(data_file, 'rb');     % binary read

if fid == -1
    fprintf(2, '❌  Unable to open "%s": %s\n', data_file, errmsg);
    return              % Abort this cell early
end
if IsComplex
    if Is16Bits
        [raw, ~] = fread(fid, 2*secondOfData*fs, 'int16');
    elseif Is8Bits
        [raw, ~] = fread(fid, 2*secondOfData*fs, 'int8');
    else
        [raw, ~] = fread(fid, 2*secondOfData*fs, 'float32');
    end
    data = raw(1:2:end) + 1i*raw(2:2:end);
else
    if Is16Bits
        [data, ~] = fread(fid, secondOfData*fs, 'int16');
    elseif Is8Bits
        [data, ~] = fread(fid, secondOfData*fs, 'int8');
    else
        [data, ~] = fread(fid, secondOfData*fs, 'float32');
    end
end
fclose(fid);

% Remove initial transient samples
if length(data) > 1e6
    data(1:1e6) = [];
end

## 4  Power Spectral Density
Use Welch's method to estimate PSD and plot it. Frequency axis is in MHz.
https://en.wikipedia.org/wiki/Welch%27s_method 

In [ ]:
disp("➡️  [PSD]  Starting Welch PSD estimate …");

% ------------------------------------------------------------
% 1. Make sure the Octave-Forge “signal” package is available
% ------------------------------------------------------------
has_signal = false;
try
    % does "signal" appear in the installed-package list?
    pkgs = pkg("list");
    for k = 1 : numel(pkgs)
        if strcmp(pkgs{k}.name, "signal")
            has_signal = true;
            break
        end
    end
catch
    % pkg() itself not found (rare)
end

if has_signal
    try
        pkg load signal
        disp("   ✓  Loaded Octave-Forge package: signal");
    catch ME
        has_signal = false;
        fprintf(2,"   ⚠️  Could not load 'signal' package (%s). Falling back to naive PSD.\n", ME.message);
    end
else
    fprintf(2,"   ⚠️  Package 'signal' not installed. Falling back to naive PSD.\n");
end

% ------------------------------------------------------------
% 2. Compute PSD
% ------------------------------------------------------------
if has_signal
    [pw, f] = pwelch(data, [], [], [], fs);    % Welch method
    disp("   ✓  PSD computed with pwelch()");
else
    % --- simple rectangular window periodogram as a fallback ---
    Nfft  = 4096;
    win   = hanning(Nfft);                     % a little windowing to reduce leakage
    S     = fftshift( fft( data(1:Nfft) .* win.' , Nfft ) );
    pw    = (abs(S).^2) / (fs * sum(win.^2));
    f     = linspace(-fs/2, fs/2, Nfft);
    disp("   ✓  PSD computed with naive FFT method");
end

% ------------------------------------------------------------
% 3. Plot
% ------------------------------------------------------------
figure;
if IsComplex
    plot((f + fl1 - fs/2)/1e6, 10*log10(pw), 'LineWidth', 1);
else
    plot(f/1e6,          10*log10(pw), 'LineWidth', 1);
end
grid on;
xlabel('Frequency (MHz)');
ylabel('PSD (dBW/Hz)');
title('Power Spectral Density');

disp("✅  [PSD]  Finished and plotted.");


### 5.  Time‑Domain View
Visualize the first 100,000 samples of the real and imaginary parts.

In [ ]:
disp("➡️  [TD]  Plotting real & imaginary parts …");

% ------------------------------------------------------------
% 1. Decide how many samples we can safely show
% ------------------------------------------------------------
if numel(data) < 1e5
    nplot = numel(data);
    fprintf(2, "   ⚠️  Only %d samples available – plotting all of them.\n", nplot);
else
    nplot = 1e5;
    fprintf("   ✓  Using first %d samples for the snapshot.\n", nplot);
end

% ------------------------------------------------------------
% 2. Real component
% ------------------------------------------------------------
figure;
plot(real(data(1:nplot)), 'r.');
xlabel('Sample');  ylabel('Amplitude');
title(sprintf('Real Part (first %d samples)', nplot));

% ------------------------------------------------------------
% 3. Imaginary component
% ------------------------------------------------------------
figure;
plot(imag(data(1:nplot)), 'b.');
xlabel('Sample');  ylabel('Amplitude');
title(sprintf('Imaginary Part (first %d samples)', nplot));

disp("✅  [TD]  Finished plotting time-domain snapshots.");


### 6. Histogram of Real Component
Plot the amplitude distribution to check for clipping or quantization.

In [ ]:
figure;
hist(real(data(1:1e5)), 18);
xlabel('Amplitude');
ylabel('Count');
title('Histogram of Real Part');

### 7.  Save Processed Data

In [ ]:
% Save to MAT-file
save('IQ_samples.mat', 'data');
disp('✅ Data saved to IQ_samples.mat');
disp('✅ Check in the left sidebar, you should find the .mat file there');
disp('⬅️ Proceed to the next notebook and click ');

## 1. Common helper functions

Below are the three MATLAB/Octave functions that every GNSS acquisition routine needs.

* **`CodeAcquisition`** – builds a 2‑D delay/Doppler search space using an FFT‑based correlation technique.
* **`NoiseVarianceEstimator`** – quick-and-dirty estimate of the noise floor $\sigma_n^2$.
* **`InverseChiSquarePfa`** – converts a desired probability of false alarm ($P_{FA}$) to a detection threshold.

🔍 ***Where you still need to work*** – The first function contains *TODO* markers so you can practise the maths behind the FFT acquisition.

In [ ]:
%% CodeAcquisition.m
function sspace = CodeAcquisition(signal, rep, Nc, Doppler_vec, fs, fi)
% Purpose.
% Evaluate the search space for the code acquisition using the time domain FFT technique
%
% Syntax.
% sspace = DftParallelCodePhaseAcquisition(sig, locC, N, dstep, DopStep)
%
% Input Parameters.
% sig       : [vector] the Galileo/GPS input signal, corrupted by Doppler
%              shift, code delay, noise and eventually interferer
% locC      : [vector] the local code replica
%
% N         : [integer] the input signal and local code length
%
% Nd        : [integer] number of Doppler bin used for the search space
%
% DopStep   : [Hz] Doppler bin width in Hz
%
% fs        : [Hz] sampling frequency
%
% fi        : [Hz] intermediate frequency
%
% Output Parameters.
% sspace       : [matrix] the search space


fif = fi/fs;    % normalized intermediate frequency
Doppler_vec_norm = Doppler_vec/fs;    % normalized Doppler step

sspace = zeros(size(Doppler_vec_norm,2), size(signal,2));

%#########################################################################
% Complete here: Compute the complex conjugate of the FFT of the replica
% Remember: 
%     - For computational efficiency we prepare the complex conjugate of the FFT of the replica only once here
% What you need to do:
%     - first step: calculate the fft of the replica rep - use the fft() function here
%     - second step: calculate the complex conjugate of the result of step 1 - use the conj() function here
%     - complete the following command below this guidance: F_rep = 
%#########################################################################



t = 0:(size(signal,2) - 1);  % time index

for i = 1:length(Doppler_vec_norm),
     fc = fif + Doppler_vec_norm(i);
%     fc = fif + (i - ceil(length(Doppler_vec_norm)/2))*Doppler_vec_norm;
    %#########################################################################
    % Complete here: Add the Doppler to the signal samples
    % Remember: 
    %     - Acquisition process is a search over Delay and Frequency. The time delay is efficiently evaluated using the fft
    %     - For the Frequency domain we need to sequentially cover all identified Doppler bins.
    %     - shifting a complex signal in the frequency domain corresponds to multiplying with a complex exponential function
    % What you need to do:
    %     - Multiply (using the .* function) with the complex exponential of (-2i*pi*fc.*t)
    %     - complete the following command below this guidance: IQ_comp = 
    %#########################################################################
    
    
    X = fft(IQ_comp);
    sspace(i,:) = ifft(X.*F_rep);
end

sspace = real(sspace.*conj(sspace));





% sspace_1ms = reshape(sspace_abs,[size(sspace,1),2046,8]);
% sspace_1ms_sum = squeeze(sum(sspace_1ms,3));
% 
% sspace_1ms_B = reshape(sspace,[size(sspace,1),2046,8]);
% sspace_1ms_sum_B = squeeze(sum(sspace_1ms_B,3)).*conj(squeeze(sum(sspace_1ms_B,3)));
% 
% 
% 
% figure;
% surf( sspace_1ms_sum, 'EdgeColor', 'none');
%       set( gca, 'FontSize', 16)
%       xlabel('Delay [samples]');
%       ylabel('Doppler');

In [ ]:
%% NoiseVarianceEstimator.m
function sigma_est = NoiseVarianceEstimator( y, fs, fc )


% Arguments:
%   y :         [vector] contains the input samples
%   fs:         [scalar] sampling frequency
%   fc:         [scalar] code rate

%
% Returns:
%   sigma_est:     [scalar] the estimated noise variance
%%

% First generate a fictitious code
clen = round( length( y ) / fs * fc );
code =  sign( rand( 1, clen ) - 0.5 );      % A bipolar random code usually has
                                            % good correlation properties

% Resample the code
loc = reshape(repmat(code,fs/fc,1),1,fs/fc*size(code,2));

% Now compute the correlators (for a single Doppler value is enough )
correlators = CodeAcquisition( y, loc, 1, 0, fs, 0 );


% Down-sample them to get uncorrelated values:
step = round( fs / fc );
correlators = correlators(1:step:end);

% Finally my noise variance estimate
sigma_est = mean( correlators ) / 2;


In [ ]:
%% InverseChiSquarePfa.m
function Th = InverseChiSquarePfa( K, PfaT )

Th = -2*log( PfaT );

Niter = 1000;
Pfa = 0;
ii = 1;

fact = factorial( K - 1 );

while abs( PfaT - Pfa ) > 1e-3 * PfaT,
    Pfa = gammainc( Th / 2, K, 'upper' );					% Compute the ccdf
    pdf = exp( -Th / 2 ) .* ( Th / 2 ).^( K - 1 ) / fact;	% Compute the pdf

    Th = Th + ( Pfa - PfaT )/ pdf;

    ii = ii + 1;
    if ii > Niter,
        break;
    end
end


## 2. Galileo E1‑C acquisition

**Goal:** detect a Galileo E1‑C pilot code in an IQ recording.

The acquisition uses a **BOC(1,1)** modulated replica.

There are two steps for you:
1. *Task&nbsp;G1* – enter the **signal parameters** (chip rate, sample rate, code period, integration time, IF).
2. *Task&nbsp;G2* – build the **BOC sub‑carrier** used to modulate the replica.

After completing both, run the `acquire_galileo_e1c.m` script immediately below to sweep delay/Doppler space and (hopefully) find the satellite.

In [ ]:
%% Task G1 – Galileo signal parameters  ❗️EDIT ME
% Fill in the fundamental parameters before you run the script.
%-------------------------------------------------------------
% Chip rate  [Hz]
fc = ;

% Sample rate [Hz]
fs = ;

% Primary‑code period [ms]
T_primary_code = ;

% Coherent integration time [s] (e.g. 0.004)
Tcoh = ;

% Intermediate frequency after down‑conversion [Hz] (0 if already at baseband)
fi = ;


In [ ]:
%% Task G2 – build the BOC(1,1) sub‑carrier  ❗️EDIT ME
% Produce a vector `boc` with +1/‑1 values at the sampling rate fs corresponding
% to one chip (8 samples at 1.023 MHz → 8 samples) and then tile it so it matches
% the PRN length you’re using.
% Example skeleton (delete once done):
%   boc_chip = [+1*ones(1,4), -1*ones(1,4)];
%   boc      = repmat(boc_chip, 1, 4092*50);  % 4 ms of data, 50 code periods
% Your code below ↓
boc = ;


In [ ]:
%% acquire_galileo_e1c.m – run after Tasks G1 & G2
% Sanity checks – stop if you forgot to run the task cells
assert(exist('fc','var') && exist('fs','var') && exist('T_primary_code','var') && exist('Tcoh','var') && exist('fi','var'),...
       '❌ Run Task G1 first and set fc/fs/T_primary_code/Tcoh/fi');
% boc is optional – only needed if you want to overlay it yourself

clc
clear;
##close all;
%=========================================================================%
% Load the prepared IQ samples                                            %
%=========================================================================%
load IQ_samples_working_e1c.mat

%=========================================================================%
% Load the PRN  code                                                  %
%=========================================================================%

load E1_codes.mat
PRN = E1C;

%=========================================================================%
% Resample the replica                                                    %
%=========================================================================%

%#########################################################################
% Complete here: Configure for Galileo E1-C signal parameters
% Remember:
%     - the lecture material from this morning does provide you with the necessary information
% What you need to do:
%     - specify the chip rate of E1-C in Hz: fc =
%     - specify the sample rate of the recorded signal in Hz, check for this the GNURadio config: fs =
%     - specify the primary code period of E1-C in ms: T_Primary code =
%#########################################################################

N_noncoh = 1; %Number of non-coherent accumulations
Tdwell = Tcoh *N_noncoh; %Total Dwell time [ms]
Nsample = Tcoh * fs; %Number of signal samples in Coherent Integration Period

PRN = repmat(PRN,1,round(Tcoh/T_primary_code));

%=========================================================================%
% Build up the replica                                                 %
%=========================================================================%
for i=1:size(PRN,1)
  Rep(i,:) = reshape(repmat(PRN(i,:),fs/fc,1),1,fs/fc*size(PRN,2));
end

%#########################################################################
% Complete here: Generate the BOC subcarrier for the replica
% Remember:
%     - Galileo E1C is modulated as a BOC(1,1) therefore we need to generate for the replica signal
%     - You need to overlay (multiply) every PRN code chip with the corresponding BOC spreading symbol
% What you need to do:
%     - Step 1: generate a single BOC spreading symbol at the correct sample rate. Hint: considering
%               the ratio of sample rate and chip rate you need to generate a BOC spreading symbol of 8 bits.
%               BOC1_1_subcarrier = ...
%     - Step 2: repeat the generated spreading symbol (BOC1_1_subcarrier) 4092 times to align with the number of chips in the replica that we are using
%               BOC1_1_subcarrier_4ms = ....
%     - Step 3: BOC1_1_subcarrier_4ms is a vecotr of size 1 x 4092*8. Now we do repeat this vector 50 times (this is the number of PRN sequences we have loaded)
%               use for this the repmat function. BOC1_1_subcarrier_4ms = ...
%     - Step 4: Now we do generate the final replica of all PRN sequences. Do a element by element multiplication (us for this the operator .* )
%               of the already existing variabl Rep and the variable BOC1_1_subcarreir_4ms
%               Rep = ....
%#########################################################################




%=========================================================================%
% Acquisition Process                                                     %
%=========================================================================%

Doppler_range = 6e3 ;
Doppler_step = round(2/(3*Tdwell));
Doppler_vec = Doppler_step:Doppler_step:Doppler_range;
Doppler_vec = [-fliplr(Doppler_vec) 0 Doppler_vec];


%=========================================================================%
% Search over all possible PRN sequences                                  %
%=========================================================================%
for code_select = 10:10
Rep_search = Rep(code_select,:);

Nc = round(Tcoh/1e-3)*fs/1e3;
sspace = 0;
for ii = 1:N_noncoh
    y =  data( (ii - 1) * Nc + (1:2*Nc) ).';   % use just 1 period of signal at the time

    % Compute the search space for a single coherent integration epoch
    Tsspace = CodeAcquisition( y, [Rep_search zeros(1,size(Rep_search,2))], Nc, Doppler_vec, fs, fi);
    sspace = sspace + Tsspace;  % Non-coherently accumulate the results

end

%=========================================================================%
% Decision Logic                                                          %
%=========================================================================%
Pfa_global = 0.1;
Pfa_cell = 1-(1-Pfa_global).^(1/numel(sspace));

Th = InverseChiSquarePfa(N_noncoh, Pfa_cell);
sigma_est = NoiseVarianceEstimator(y, fs, fc);
Th = sigma_est * Th;

##sspace_single_period = reshape(sspace,[size(sspace,1),2*fs/fc*size(PRN,2)/(Tcoh*1e3),2*Tcoh*1e3]);
##sspace_single_period = squeeze(sum(sspace_single_period,3));


if (max(max(sspace)) > Th)
       [doppler_id, delay_id] = find(sspace == max(max(sspace)));
       fprintf('PRN %i acquired, Doppler [Hz]: %f, Code Delay [samples]: %i\n',code_select, Doppler_vec(doppler_id(1)), delay_id(1));
        figure
          surf( sspace, 'EdgeColor', 'none');
          set( gca, 'FontSize', 16)
          xlabel('Delay [samples]');
          ylabel('Doppler Bin');
          xlim([0 size(sspace,2)]);
          ylim([1 length(Doppler_vec)])

end


end




## 3. GPS L1 C/A acquisition

**Goal:** detect a legacy GPS C/A code.

Two open tasks:
1. *Task&nbsp;P1* – up‑sample the C/A PRN replica to the 2.046 MHz sample rate.
2. *Task&nbsp;P2* – build a Doppler search vector covering ±10 kHz.

When you’re done, run `acquire_gps_ca.m` to execute the search.

In [ ]:
%% Task P1 – up‑sample the GPS C/A code  ❗️EDIT ME
% Build the matrix `Rep` so that each row i contains the up‑sampled replica
% for PRN i.
% Hint: use `repmat` and `reshape` to repeat every chip twice (1.023 → 2.046 MHz).
Rep = ;


In [ ]:
%% Task P2 – Doppler search vector  ❗️EDIT ME
% Create a row vector `Doppler_vec` that goes from –10 kHz to +10 kHz in steps of ~667 Hz.
% The exact endpoints don''t matter as long as 0 Hz is in the middle.
Doppler_vec = ;


In [ ]:
%% acquire_gps_ca.m – run after Tasks P1 & P2
% Sanity checks
assert(exist('Rep','var') && exist('Doppler_vec','var'), '❌ Run Task P1/P2 first');

clc
clear;
##close all;
%=========================================================================%
% Load the prepared IQ samples                                            %
%=========================================================================%
load IQ_samples.mat

%=========================================================================%
% Load the PRN  code                                                  %
%=========================================================================%

load gps_ca_codes.mat;

%=========================================================================%
% Resample the replica                                                    %
%=========================================================================%

fc = 1.023e6; % GNSS signal chip rate
fs = 2.046e6; % Sample rate
Tcoh = 5e-3; % Coherent Integration time [ms]
T_primary_code = 1e-3; %Period of the primary code [ms]
N_noncoh = 1; %Number of non-coherent accumulations
Tdwell = Tcoh *N_noncoh; %Total Dwell time [ms]
Nsample = Tcoh * fs; %Number of signal samples in Coherent Integration Period

PRN = repmat(gps_ca,1,round(Tcoh/T_primary_code));

%=========================================================================%
% Build up the replica                                                 %
%=========================================================================%
for i=1:size(PRN,1)
%#########################################################################
% Complete here: Build up the replica
% Remember: 
%     - the GPS C/A is a BPSK(1), i.e. a signal at a rate of 1.023 MHz
%     - the recorded signal is sampled at a rate of 2.046 MHz
% What you need to do:
%     - up-sample the C/A code replica to 2.046 MHz, i.e. repeat every chip of the C/A code replica 2 times
%     - you can use the functions repmat and reshape (ref. homework)
%     - complete the following command below this guidance: Rep(i,:) = 
%#########################################################################
 
end


%=========================================================================%
% Acquisition Process                                                     %
%=========================================================================%

Doppler_range = 10e3 ;
Doppler_step = round(2/(3*Tdwell));

%#########################################################################
% Complete here: Set up the Doppler Vector
% Remember: 
%     - Doppler for a static user can be in the range [-10 kHz ... 10 kHz] (with good margin)
% What you need to do:
%     - build up a vector (single row, N columns) that ranges from -Doppler_range in steps of Doppler_step up to +Doppler_range
%     - for symmetry purpuses it is good to center around 0 Hz, you do not need to match exactly +/- 10 kHz on the lower/upper limit
%     - complete the following command below this guidance: Doppler_vec = 
%#########################################################################




%=========================================================================%
% Search over all possible PRN sequences                                  %
%=========================================================================%
for code_select = 1:20
Rep_search = Rep(code_select,:);

Nc = round(Tcoh/1e-3)*fs/1e3;
sspace = 0;
for ii = 1:N_noncoh
    y =  data( (ii - 1) * Nc + (1:Nc) ).';   % use just 1 period of signal at the time

    % Compute the search space for a single coherent integration epoch
    Tsspace = CodeAcquisition( y, Rep_search, Nc, Doppler_vec, fs, fi);
    sspace = sspace + Tsspace;  % Non-coherently accumulate the results

end

%=========================================================================%
% Decision Logic                                                          %
%=========================================================================%
Pfa_global = 0.1;
Pfa_cell = 1-(1-Pfa_global).^(1/numel(sspace));

Th = InverseChiSquarePfa(N_noncoh, Pfa_cell);
sigma_est = NoiseVarianceEstimator(y, fs, fc);
Th = sigma_est * Th;

sspace_single_period = reshape(sspace,[size(sspace,1),fs/fc*size(PRN,2)/(Tcoh*1e3),Tcoh*1e3]);
sspace_single_period = squeeze(sum(sspace_single_period,3));


if (max(max(sspace)) > Th)
       [doppler_id, delay_id] = find(sspace == max(max(sspace)));
       fprintf('PRN %i acquired, Doppler [Hz]: %f, Code Delay [samples]: %i\n',code_select, Doppler_vec(doppler_id(1)), delay_id(1));
        figure
          surf( sspace_single_period, 'EdgeColor', 'none');
          set( gca, 'FontSize', 16)
          xlabel('Delay [samples]');
          ylabel('Doppler Bin');
          xlim([0 size(sspace_single_period,2)]);
          ylim([1 length(Doppler_vec)])

end


end


